# Symbol Class

This Symbol class provides a basic interface to interact with all the information regarding a given symbol. It contains a set of Velas objects with the symbol's candlestick data at different timeframes. It also provides a similar set functionalities to the Velas class:
<ul>   
    <li> Loading and saving the raw candlestick data from different sources (.csv, SQL..).</li>
    <li> Handling the data updating, missing data, separating trading sessions...</li>
    <li> Technical Analysis indicators</li>
    <li> Common plots.</li>
</ul>    


### Import libraries
We import the needed standard libraries first. Then we add the directory above path to the system so that when import the library, it will load the local version in the folder, instead of the version installed with pip. Finally we load the classes, functions and enums from the library, which we are going to use.

In [1]:
import datetime as dt
import pandas as pd
from IPython.display import Image
%matplotlib qt 

from traphing.data_classes import Velas, Symbol
from traphing.utils import Timeframes, unwrap
from traphing.graph.Gl import gl

In [33]:
storage_folder = "../tests/data/storage"
storage_folder2 = "../tests/data/storage2"
updates_folder = "./MT5/MQL5/MQL5/Files/"

## Creating a Symbol instance

When we create a Symbol object, we need to specify the basic parameters of the symbol data:
 - <b>symbol_name</b>: The name of the symbol given by the broker we are using (it can be different from broker to broker)
 -  <b>timeframes</b>: A list with the different timeframes of the candlesticks we want to include.

No candlestick data is loaded when creating the object.

In [17]:
symbol_name = "AUDCHF"
timeframes = [Timeframes.M15, Timeframes.D1]
symbol = Symbol(symbol_name, timeframes)

print (type(symbol))

<class 'traphing.data_classes._Symbol._Symbol.Symbol'>


The Velas objects are initialized with no candlestick data. They are stored in an internal dictionary where the key is the timeframe and the value is the Velas object. This implementation should be transparent to the usage of the Symbol class. 

We can access the *Velas* objects inside the *Symbol* instance by their timeframe. We can get the current timeframes in the Symbol object by means of the attribute *timeframes* which has been modify using a *getter* to compute the list dinamically when accessed.

We can use the *Symbol* object as a dictionary to get the *Velas* objects.

In [18]:
symbol.timeframes

[<Timeframes.M15: 15>, <Timeframes.D1: 1440>]

In [19]:
symbol[Timeframes.M15]

### Adding and removing Velas objects

We can easily add, and delete velas inside the object using the following methods.

In [20]:
symbol.add_velas(Timeframes.M5, None)
print (symbol.timeframes)

[<Timeframes.M15: 15>, <Timeframes.D1: 1440>, <Timeframes.M5: 5>]


In [21]:
symbol.del_velas(Timeframes.M5)
print (symbol.timeframes)

[<Timeframes.M15: 15>, <Timeframes.D1: 1440>]


## Loading the data from .csv

The Symbol class makes it easy to load the candlestick data for all the timeframes at once. It has the same loading functions as the Velas class, which internally call the same function for all of its Velas objects.

In [22]:
symbol.load_data_from_csv(storage_folder)

## Set interval

We can set the time interval directly to the symbol object using the same interface as the one for Velas. The interval applied to the *Symbol* is applied to all of its Velas objects.

In [23]:
start_time = dt.datetime(2019,7,20)
end_time = dt.datetime(2019,8,20)

symbol.set_time_interval(start_time, end_time, trim = False)

In [24]:
symbol[Timeframes.M15].df.head(5)

,Open,High,Low,Close,Volume
Timestamp,,,,,
2019-07-22 00:00:00,0.69208,0.69241,0.69173,0.69213,58.0
2019-07-22 00:15:00,0.69208,0.69240,0.69124,0.69203,187.0
2019-07-22 00:30:00,0.69209,0.69253,0.69167,0.69171,411.0
2019-07-22 00:45:00,0.69199,0.69232,0.69143,0.69185,182.0
2019-07-22 01:00:00,0.69209,0.69222,0.69142,0.69153,267.0


In [25]:
symbol[Timeframes.D1].df.head(5)

,Open,High,Low,Close,Volume
Timestamp,,,,,
2019-07-22,0.69208,0.69299,0.69035,0.69067,45803.0
2019-07-23,0.69067,0.69213,0.68852,0.68991,52588.0
2019-07-24,0.68991,0.69014,0.68677,0.68678,54249.0
2019-07-25,0.68680,0.69041,0.68601,0.68844,67797.0
2019-07-26,0.68854,0.68894,0.68565,0.68609,48966.0


## Symbol properties

There is a set of information that we need from the symbol.

This information is currently obtained from the MT5, storing it in a csv file with name: 
- symbols_properties.csv


In [26]:
symbols_properties_df = Symbol.load_symbols_properties_csv(storage_folder)
symbols_properties_df

,point_size,min_tick_value,contract_size,currency
symbol_name,,,,
EURUSD,0.00001,1.000000,100000.0,EUR
GBPUSD,0.00001,1.000000,100000.0,GBP
USDCHF,0.00001,1.005288,100000.0,USD
USDJPY,0.00100,0.920116,100000.0,USD
USDCNH,0.00001,0.141807,100000.0,USD
...,...,...,...,...
NZDCHF,0.00001,1.005288,100000.0,NZD
NZDJPY,0.00100,0.920116,100000.0,NZD
NZDSGD,0.00001,0.733337,100000.0,NZD


We can also save the file to disk:

In [28]:
Symbol.save_symbols_properties_to_csv(storage_folder2,symbols_properties_df)

In order to load the information to the Symbol's SymbolProperties object, we use the method:

In [29]:
symbol.load_properties_from_df(symbols_properties_df)
unwrap(symbol.properties)

<SymbolProperties>	object has children:
    <str>	symbol_name:	AUDCHF
    <NoneType>	type:	None
    <NoneType>	country:	None
    <str>	currency:	AUD
    <NoneType>	sector:	None
    <float>	contract_size:	100000.0
    <float>	point_size:	1e-05
    <float>	min_tick_value:	1.00528781




## Market Hours

TODO: Explained class its own notebook.
The Symbol class contains an instance and interface for the most common operations such as:
- Estimating the open and close Market hours.
- Obtaining the samples organized by trading sessions.

In [30]:
symbol.estimate_market_hours()

## Unwrapping the object

The following shows the unwrap of a Symbol object. In it, you can appreciate that:
 - The start and end time of the interval are not stored at *Symbol* level, but only at *Velas* level. This allows different timesticks to have different intervals.
 - The properties object contains all of the static information regarding the symbol, except for the market hours.
 - The MarketHours object contains the information about open market hours

In [31]:
unwrap(symbol, "symbol")

<Symbol>	symbol has children:
    <str>	symbol_name:	AUDCHF
    <dict>	_velas_dict
    <SymbolProperties>	properties
    <MarketHours>	market_hours
    <datetime>	start_time:	2019-07-20 00:00:00
    <datetime>	end_time:	2019-08-20 00:00:00

  <MarketHours>	market_hours has children:
      <time>	open_time:	00:00:00
      <time>	close_time:	00:00:00
      <list>	trading_days_list
      <dict>	special_days_dict

    <dict>	special_days_dict has children:

    <list>	trading_days_list has children:
        <int>	trading_days_list[0]:	0
        <int>	trading_days_list[1]:	1
        <int>	trading_days_list[2]:	2
        <int>	trading_days_list[3]:	3
        <int>	trading_days_list[4]:	4

  <SymbolProperties>	properties has children:
      <str>	symbol_name:	AUDCHF
      <NoneType>	type:	None
      <NoneType>	country:	None
      <str>	currency:	AUD
      <NoneType>	sector:	None
      <float>	contract_size:	100000.0
      <float>	point_size:	1e-05
      <float>	min_tick_value:	1.00528781

  <

## Indicators

The Symbol class only implements indicators that need several candlestick data at timeframes to be computed. In order to obtain indicators for a given timeframe, use the Velas objects directly. 

NO INDICATORS IMPLEMENTED SO FAR

The interface for accessing

## Plottings

Like with the indicators, we only implement plotting functions that require different timeframes.

NO IMPLEMENTATION SO FAR